<a href="https://colab.research.google.com/github/Marcfeitosa/Data-Science-Projects/blob/main/C%C3%B3pia_de_Sports_Bettor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone the github repository

In [ ]:
#Remove Preexisting Files
! rm -rf NBA-Machine-Learning-Sports-Betting
! rm -rf *

#Bootstrap Files
! git clone https://github.com/kyleskom/NBA-Machine-Learning-Sports-Betting.git
! mv -v ./NBA-Machine-Learning-Sports-Betting/* .
! pip3 install -r requirements.txt

#Clear Bootstrap Logs
from IPython.display import clear_output
clear_output()

print("Successful Bootstrap!!!")

Successful Bootstrap!!!


## Runs models XGBoost and Neural networks

In [ ]:
!python3 main.py -A -odds=fanduel

2023-04-26 18:02:07.212860: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 18:02:08.969096: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-04-26 18:02:08.969330: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-04-26 18:02:08.969359: W tensorfl

### Transform the XGBoost text into Data Frame

In [ ]:
# Changes the XGBoost pandas into a Data Frame

import re
import pandas as pd

text = '''Cleveland Cavaliers (64.9%) vs New York Knicks: UNDER 202.5 (58.5%)
Memphis Grizzlies (71.1%) vs Los Angeles Lakers: UNDER 222.5 (55.8%)
Milwaukee Bucks (68.3%) vs Miami Heat: OVER 220 (53.1%)
Sacramento Kings (61.4%) vs Golden State Warriors: OVER 234.5 (67.0%)'''

rows = []
for d in text.split('\n'):
    match = re.search(r'(.+?)\s*(\(.+?\))?\s*vs\s*(.+?)\s*(\(.+?\))?:\s*(.+?)\s*(?P<over_under>OVER|UNDER)\s+(?P<value>\d+(\.\d+)?)\s*\((?P<percentage>\d+(\.\d+)?%)\)', d)
    if match:
        team1 = match.group(1).strip()
        percentteam1 = match.group(2).strip('()') if match.group(2) else ''
        team2 = match.group(3).strip()
        percentteam2 = match.group(4).strip('()') if match.group(4) else ''
        over_under = match.group('over_under')
        value = match.group('value')
        percentage = match.group('percentage')
        rows.append([team1, percentteam1, team2, percentteam2, over_under, value, percentage])

dfXGBoost = pd.DataFrame(rows, columns=['Team 1', 'Team Percentage 1', 'Team 2', 'Team Percentage 2', 'Over/Under', 'Value', 'Over/Under Percentage'])
dfXGBoost['Team Percentage 1'] = dfXGBoost.apply(lambda row: f"{float(row['Team Percentage 1'].strip('%')):.2f}%" if row['Team Percentage 1'] else f"{100 - float(row['Team Percentage 2'].strip('%')):.2f}%", axis=1)
dfXGBoost['Team Percentage 2'] = dfXGBoost.apply(lambda row: f"{float(row['Team Percentage 2'].strip('%')):.2f}%" if row['Team Percentage 2'] else f"{100 - float(row['Team Percentage 1'].strip('%')):.2f}%", axis=1)
dfXGBoost['Over/Under Percentage'] = dfXGBoost['Over/Under Percentage'].apply(lambda x: f"{float(x.strip('%')):.2f}%")
print(dfXGBoost)


                Team 1 Team Percentage 1                 Team 2  \
0  Cleveland Cavaliers            64.90%        New York Knicks   
1    Memphis Grizzlies            71.10%     Los Angeles Lakers   
2      Milwaukee Bucks            68.30%             Miami Heat   
3     Sacramento Kings            61.40%  Golden State Warriors   

  Team Percentage 2 Over/Under  Value Over/Under Percentage  
0            35.10%      UNDER  202.5                58.50%  
1            28.90%      UNDER  222.5                55.80%  
2            31.70%       OVER    220                53.10%  
3            38.60%       OVER  234.5                67.00%  


In [ ]:
dfXGBoost

,Team 1,Team Percentage 1,Team 2,Team Percentage 2,Over/Under,Value,Over/Under Percentage
0,Cleveland Cavaliers,64.90%,New York Knicks,35.10%,UNDER,202.5,58.50%
1,Memphis Grizzlies,71.10%,Los Angeles Lakers,28.90%,UNDER,222.5,55.80%
2,Milwaukee Bucks,68.30%,Miami Heat,31.70%,OVER,220,53.10%
3,Sacramento Kings,61.40%,Golden State Warriors,38.60%,OVER,234.5,67.00%


In [ ]:
# Percentage reduction due to sentiment analisys in XGBoos Home team

dfXGBoost_reduction1 = dfXGBoost[['Team 1', 'Team Percentage 1']]
dfXGBoost_reduction1

,Team 1,Team Percentage 1
0,Cleveland Cavaliers,64.90%
1,Memphis Grizzlies,71.10%
2,Milwaukee Bucks,68.30%
3,Sacramento Kings,61.40%


In [ ]:
# Percentage reduction due to sentiment analisys in XGBoos Away team

dfXGBoost_reduction2 = dfXGBoost[['Team 2', 'Team Percentage 2']]
dfXGBoost_reduction2

,Team 2,Team Percentage 2
0,New York Knicks,35.10%
1,Los Angeles Lakers,28.90%
2,Miami Heat,31.70%
3,Golden State Warriors,38.60%


In [ ]:
# merge the two data frames horizontally
merged_df_XGBoost = pd.concat([dfXGBoost_reduction1, dfXGBoost_reduction2.rename(columns={'Team 2': 'Team 1', 'Team Percentage 2': 'Team Percentage 1'})], ignore_index=True)

# sort the merged data frame by the team name
merged_df_XGBoost = merged_df_XGBoost.sort_values('Team 1').reset_index(drop=True)

# change the names of the columns
new_columns = {
    'Team 1': 'Team',
    'Team Percentage 1': 'Team Percentage'
}

# rename the columns in the merged data frame
merged_df_XGBoost = merged_df_XGBoost.rename(columns=new_columns)

# display the merged data frame
merged_df_XGBoost

,Team,Team Percentage
0,Cleveland Cavaliers,64.90%
1,Golden State Warriors,38.60%
2,Los Angeles Lakers,28.90%
3,Memphis Grizzlies,71.10%
4,Miami Heat,31.70%
5,Milwaukee Bucks,68.30%
6,New York Knicks,35.10%
7,Sacramento Kings,61.40%


### Changes the Neural networks model into Data Frame

In [ ]:
# Changes the Neural Networks pandas into a Data Frame

import re
import pandas as pd

text = '''Cleveland Cavaliers (64.9%) vs New York Knicks: OVER 202.5 (52.2%)
Memphis Grizzlies (72.9%) vs Los Angeles Lakers: OVER 222.5 (50.5%)
Milwaukee Bucks (74.1%) vs Miami Heat: OVER 220 (50.3%)
Sacramento Kings (63.7%) vs Golden State Warriors: OVER 234.5 (51.8%)'''

rows = []
for d in text.split('\n'):
    match = re.search(r'(.+?)\s*(\(.+?\))?\s*vs\s*(.+?)\s*(\(.+?\))?:\s*(.+?)\s*(?P<over_under>OVER|UNDER)\s+(?P<value>\d+(\.\d+)?)\s*\((?P<percentage>\d+(\.\d+)?%)\)', d)
    if match:
        team1 = match.group(1).strip()
        percentteam1 = match.group(2).strip('()') if match.group(2) else ''
        team2 = match.group(3).strip()
        percentteam2 = match.group(4).strip('()') if match.group(4) else ''
        over_under = match.group('over_under')
        value = match.group('value')
        percentage = match.group('percentage')
        rows.append([team1, percentteam1, team2, percentteam2, over_under, value, percentage])

dfNeuralNetworks = pd.DataFrame(rows, columns=['Team 1', 'Team Percentage 1', 'Team 2', 'Team Percentage 2', 'Over/Under', 'Value', 'Over/Under Percentage'])
dfNeuralNetworks['Team Percentage 1'] = dfNeuralNetworks.apply(lambda row: f"{float(row['Team Percentage 1'].strip('%')):.2f}%" if row['Team Percentage 1'] else f"{100 - float(row['Team Percentage 2'].strip('%')):.2f}%", axis=1)
dfNeuralNetworks['Team Percentage 2'] = dfNeuralNetworks.apply(lambda row: f"{float(row['Team Percentage 2'].strip('%')):.2f}%" if row['Team Percentage 2'] else f"{100 - float(row['Team Percentage 1'].strip('%')):.2f}%", axis=1)
dfNeuralNetworks['Over/Under Percentage'] = dfNeuralNetworks['Over/Under Percentage'].apply(lambda x: f"{float(x.strip('%')):.2f}%")
print(dfNeuralNetworks)


                Team 1 Team Percentage 1                 Team 2  \
0  Cleveland Cavaliers            64.90%        New York Knicks   
1    Memphis Grizzlies            72.90%     Los Angeles Lakers   
2      Milwaukee Bucks            74.10%             Miami Heat   
3     Sacramento Kings            63.70%  Golden State Warriors   

  Team Percentage 2 Over/Under  Value Over/Under Percentage  
0            35.10%       OVER  202.5                52.20%  
1            27.10%       OVER  222.5                50.50%  
2            25.90%       OVER    220                50.30%  
3            36.30%       OVER  234.5                51.80%  


In [ ]:
dfNeuralNetworks

,Team 1,Team Percentage 1,Team 2,Team Percentage 2,Over/Under,Value,Over/Under Percentage
0,Cleveland Cavaliers,64.90%,New York Knicks,35.10%,OVER,202.5,52.20%
1,Memphis Grizzlies,72.90%,Los Angeles Lakers,27.10%,OVER,222.5,50.50%
2,Milwaukee Bucks,74.10%,Miami Heat,25.90%,OVER,220,50.30%
3,Sacramento Kings,63.70%,Golden State Warriors,36.30%,OVER,234.5,51.80%


In [ ]:
# Percentage reduction due to sentiment analisys in XGBoos Home team

dfNeuralNetworks_reduction1 = dfNeuralNetworks[['Team 1', 'Team Percentage 1']]
dfNeuralNetworks_reduction1

,Team 1,Team Percentage 1
0,Cleveland Cavaliers,64.90%
1,Memphis Grizzlies,72.90%
2,Milwaukee Bucks,74.10%
3,Sacramento Kings,63.70%


In [ ]:
dfNeuralNetworks_reduction2 = dfNeuralNetworks[['Team 2', 'Team Percentage 2']]
dfNeuralNetworks_reduction2

,Team 2,Team Percentage 2
0,New York Knicks,35.10%
1,Los Angeles Lakers,27.10%
2,Miami Heat,25.90%
3,Golden State Warriors,36.30%


In [ ]:
# merge the two data frames horizontally
merged_df_dfNeuralNetworks = pd.concat([dfNeuralNetworks_reduction1, dfNeuralNetworks_reduction2.rename(columns={'Team 2': 'Team 1', 'Team Percentage 2': 'Team Percentage 1'})], ignore_index=True)

# sort the merged data frame by the team name
merged_df_dfNeuralNetworks = merged_df_dfNeuralNetworks.sort_values('Team 1').reset_index(drop=True)

# change the names of the columns
new_columns = {
    'Team 1': 'Team',
    'Team Percentage 1': 'Team Percentage'
}

# rename the columns in the merged data frame
merged_df_dfNeuralNetworks = merged_df_dfNeuralNetworks.rename(columns=new_columns)

# display the merged data frame
merged_df_dfNeuralNetworks

,Team,Team Percentage
0,Cleveland Cavaliers,64.90%
1,Golden State Warriors,36.30%
2,Los Angeles Lakers,27.10%
3,Memphis Grizzlies,72.90%
4,Miami Heat,25.90%
5,Milwaukee Bucks,74.10%
6,New York Knicks,35.10%
7,Sacramento Kings,63.70%


# Twiter part (analisys sentiment)

In [1]:
# Scraping Twitter for all the analisy sentiment
!pip install snscrape

from tqdm.notebook import tqdm
import snscrape.modules.twitter as snstwitter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 3.3 MB/s eta 0:00:00


In [2]:
# create a dictionary to map team names to their corresponding hashtags
team_dict = {'Atlanta Hawks': '#TrueToAtlanta',
             'Boston Celtics': '#BleedGreen',
             'Brooklyn Nets': '#NetsWorld',
             'Charlotte Hornets': '#LetsFly',
             'Chicago Bulls': '#BullsNation',
             'Cleveland Cavaliers': '#LetEmKnow',
             'Dallas Mavericks': '#MFFL',
             'Denver Nuggets': '#MileHighBasketball',
             'Detroit Pistons': '#Pistons',
             'Golden State Warriors': '#DubNation',
             'Houston Rockets': '#Rockets',
             'Indiana Pacers': '#BoomBaby',
             'LA Clippers': '#ClipperNation',
             'Los Angeles Lakers': '#LakeShow',
             'Memphis Grizzlies': '#BigMemphis',
             'Miami Heat': '#HeatCulture',
             'Milwaukee Bucks': '#FearTheDeer',
             'Minnesota Timberwolves': '#RaisedByWolves',
             'New Orleans Pelicans': '#Pelicans',
             'New York Knicks': '#NewYorkForever',
             'Oklahoma City Thunder': '#ThunderUp',
             'Orlando Magic': '#MagicTogether',
             'Philadelphia 76ers': '#BrotherlyLove',
             'Phoenix Suns': '#WeAreTheValley',
             'Portland Trail Blazers': '#RipCity',
             'Sacramento Kings': '#SacramentoProud',
             'San Antonio Spurs': '#PorVida',
             'Toronto Raptors': '#WeTheNorth',
             'Utah Jazz': '#TakeNote',
             'Washington Wizards': '#DCAboveAll'}

# get the list of teams playing today from the merged_df_XGBoost dataframe
today_teams = list(set(merged_df_XGBoost['Team']))

# filter the team_dict dictionary to get only the hashtags for the teams playing today
today_hashtags = {k: v for k, v in team_dict.items() if k in today_teams}

# get a list of hashtags for the teams playing today
today_hashtags_list = list(today_hashtags.values())

# set the number of tweets to retrieve
n_tweets = 1000

tweets = []

for hashtag in today_hashtags_list:
    hashtag_str = hashtag
    scraper = snstwitter.TwitterSearchScraper(hashtag_str)

    for i, tweet in tqdm(enumerate(scraper.get_items()), total=n_tweets):
        data = [
            tweet.date.today,
            tweet.rawContent,
            tweet.likeCount,
            tweet.retweetCount,
            hashtag_str
        ]
        tweets.append(data)
        if i >= n_tweets:
            break

# create a dataframe from the tweets
tweet_df = pd.DataFrame(tweets, columns=['date', 'rawContent', 'likecount', 'retweetCount', 'hashtags'])


NameError: ignored

In [ ]:
tweet_df

,date,rawContent,likecount,retweetCount,hashtags
0,<built-in method today of type object at 0x96e...,How to watch Game 2 of the #Timberwolves vs. #...,0,0,#MileHighBasketball
1,<built-in method today of type object at 0x96e...,"Welcome back to playoff basketball 🏀, Jamal! 👊...",0,0,#MileHighBasketball
2,<built-in method today of type object at 0x96e...,Nuggets cruise to easy Game 1 victory over Tim...,0,0,#MileHighBasketball
3,<built-in method today of type object at 0x96e...,🙋‍♂️ #MileHighBasketball,0,0,#MileHighBasketball
4,<built-in method today of type object at 0x96e...,It’s game day baby !!! #BringItIn #MileHighBas...,0,0,#MileHighBasketball
...,...,...,...,...,...
6001,<built-in method today of type object at 0x96e...,AND THAT WILL DO IT!!!!!! \n\nTimberwolves adv...,0,0,#RaisedByWolves
6002,<built-in method today of type object at 0x96e...,Dos años consecutivos en Playoffs. Que sea el ...,0,0,#RaisedByWolves
6003,<built-in method today of type object at 0x96e...,WOLVES IN 6. #RaisedByWolves,0,0,#RaisedByWolves
6004,<built-in method today of type object at 0x96e...,Karl-Anthony Towns put on a show as the Timber...,1,0,#RaisedByWolves


Applying sentiment analisys

In [ ]:
!pip install numpy --upgrade

import pandas as pd
from textblob import TextBlob

# create a dictionary to map team names to their corresponding hashtags
team_dict = {'Atlanta Hawks': '#TrueToAtlanta',
             'Boston Celtics': '#BleedGreen',
             'Brooklyn Nets': '#NetsWorld',
             'Charlotte Hornets': '#LetsFly',
             'Chicago Bulls': '#BullsNation',
             'Cleveland Cavaliers': '#LetEmKnow',
             'Dallas Mavericks': '#MFFL',
             'Denver Nuggets': '#MileHighBasketball',
             'Detroit Pistons': '#Pistons',
             'Golden State Warriors': '#DubNation',
             'Houston Rockets': '#Rockets',
             'Indiana Pacers': '#BoomBaby',
             'LA Clippers': '#ClipperNation',
             'Los Angeles Lakers': '#LakeShow',
             'Memphis Grizzlies': '#BigMemphis',
             'Miami Heat': '#HeatCulture',
             'Milwaukee Bucks': '#FearTheDeer',
             'Minnesota Timberwolves': '#RaisedByWolves',
             'New Orleans Pelicans': '#Pelicans',
             'New York Knicks': '#NewYorkForever',
             'Oklahoma City Thunder': '#ThunderUp',
             'Orlando Magic': '#MagicTogether',
             'Philadelphia 76ers': '#BrotherlyLove',
             'Phoenix Suns': '#WeAreTheValley',
             'Portland Trail Blazers': '#RipCity',
             'Sacramento Kings': '#SacramentoProud',
             'San Antonio Spurs': '#PorVida',
             'Toronto Raptors': '#WeTheNorth',
             'Utah Jazz': '#TakeNote',
             'Washington Wizards': '#DCAboveAll'}

# create three counters for positive, negative, and neutral tweets
positive_tweets = {}
negative_tweets = {}
neutral_tweets = {}

# initialize the counters for each team to zero
for team in team_dict.keys():
    positive_tweets[team] = 0
    negative_tweets[team] = 0
    neutral_tweets[team] = 0

# iterate over each tweet in the dataframe
for index, row in tweet_df.iterrows():
    # get the team hashtag for the tweet
    team_hashtag = row['hashtags']
    # get the team name for the tweet
    team_name = list(team_dict.keys())[list(team_dict.values()).index(team_hashtag)]
    # create a TextBlob object for the tweet
    blob = TextBlob(row['rawContent'])
    # get the polarity score for the tweet (-1 to 1)
    polarity = blob.sentiment.polarity
    # classify the tweet as positive, negative, or neutral based on the polarity score
    if polarity > 0:
        positive_tweets[team_name] += 1
    elif polarity < 0:
        negative_tweets[team_name] += 1
    else:
        neutral_tweets[team_name] += 1

# create a new data frame to store the results
result_df = pd.DataFrame({'Team': list(team_dict.keys()),
                          'positive_tweets': list(positive_tweets.values()),
                          'negative_tweets': list(negative_tweets.values()),
                          'neutral_tweets': list(neutral_tweets.values())})

# Calculate the total tweets for each team
total_tweets = {}
for team_name in positive_tweets:
    total_tweets[team_name] = positive_tweets[team_name] + negative_tweets[team_name] + neutral_tweets[team_name]

# Calculate the percentage of positive tweets for each team
positive_percentages = {}
for team_name in positive_tweets:
    if total_tweets[team_name] != 0:
        positive_percentages[team_name] = (positive_tweets[team_name] / total_tweets[team_name]) * 100
    else:
        positive_percentages[team_name] = 0

# Add a column to the dataframe with the percentage of positive tweets
result_df['positive_percentage'] = result_df['Team'].map(positive_percentages)

today_teams = list(set(merged_df_XGBoost['Team']))

# filter the result_df to include only teams playing today
result_df = result_df[result_df['Team'].isin(today_teams)]

# print the results
result_df

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 47.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.2
    Uninstalling numpy-1.24.2:
      Successfully uninstalled numpy-1.24.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.


RuntimeError: ignored

RuntimeError: ignored

RuntimeError: ignored

RuntimeError: ignored

NameError: ignored

In [ ]:
#converting string to number
result_df['positive_percentage'] = result_df['positive_percentage'].astype(str).str.replace('%', '').astype(float)

#calculating the mean
mean_percentage_positive_tweets = result_df['positive_percentage'].mean()
mean_percentage_positive_tweets

NameError: ignored

In [ ]:
# final sentiment analisys percentage

result_df['final_sentiment'] = result_df['positive_percentage'].apply(lambda x: float(x[:-1]) - mean_percentage_positive_tweets if type(x) == str else x - mean_percentage_positive_tweets)
result_df = result_df.reset_index()
result_df

,index,Team,positive_tweets,negative_tweets,neutral_tweets,positive_percentage,final_sentiment
0,7,Denver Nuggets,312,191,498,31.168831,2.530803
1,13,Los Angeles Lakers,251,231,519,25.074925,-3.563104
2,14,Memphis Grizzlies,254,190,557,25.374625,-3.263403
3,15,Miami Heat,301,185,515,30.069930,1.431901
4,16,Milwaukee Bucks,300,286,415,29.970030,1.332001
5,17,Minnesota Timberwolves,302,198,501,30.169830,1.531802


In [ ]:
# create a new column with the difference
#merged_df_XGBoost['Sentiment_analisys_applied'] = merged_df_XGBoost['Team Percentage'].apply(lambda x: float(x[:-1])) + result_df['final_sentiment']
merged_df_XGBoost['Sentiment_analisys_applied'] = ((merged_df_XGBoost['Team Percentage'].apply(lambda x: float(x[:-1])) + result_df['final_sentiment']) / 100) *100

# add the column 'final_sentiment' to the merged data frame
#merged_df_XGBoost['final_sentiment'] = result_df['final_sentiment']
merged_df_XGBoost['final_sentiment'] = (result_df['final_sentiment'] / 100) * 100


# add columns 'Over/Under', 'Value', 'Over/Under Percentage' from the XGBoost model
merged_df_XGBoost[['Over/Under', 'Value', 'Over/Under Percentage']] = dfXGBoost[['Over/Under', 'Value', 'Over/Under Percentage']]

# print the new dataframe
merged_df_XGBoost


,Team,Team Percentage,Sentiment_analisys_applied,final_sentiment,Over/Under,Value,Over/Under Percentage
0,Denver Nuggets,72.10%,74.630803,2.530803,UNDER,226.5,55.80%
1,Los Angeles Lakers,28.90%,25.336896,-3.563104,OVER,219,53.10%
2,Memphis Grizzlies,71.10%,67.836597,-3.263403,OVER,222,64.90%
3,Miami Heat,31.70%,33.131901,1.431901,NaN,NaN,NaN
4,Milwaukee Bucks,68.30%,69.632001,1.332001,NaN,NaN,NaN
5,Minnesota Timberwolves,27.90%,29.431802,1.531802,NaN,NaN,NaN


In [ ]:
# create a new column with the difference
merged_df_dfNeuralNetworks['Sentiment_analisys_applied'] = ((merged_df_dfNeuralNetworks['Team Percentage'].apply(lambda x: float(x[:-1])) + result_df['final_sentiment']) / 100) * 100

# add the column 'final_sentiment' to the merged data frame
merged_df_dfNeuralNetworks['final_sentiment'] = (result_df['final_sentiment'] / 100) * 100

# add columns 'Over/Under', 'Value', 'Over/Under Percentage' from the XGBoost model
merged_df_dfNeuralNetworks[['Over/Under', 'Value', 'Over/Under Percentage']] = dfNeuralNetworks[['Over/Under', 'Value', 'Over/Under Percentage']]

# print the new dataframe
merged_df_dfNeuralNetworks


,Team,Team Percentage,Sentiment_analisys_applied,final_sentiment,Over/Under,Value,Over/Under Percentage
0,Denver Nuggets,74.90%,77.430803,2.530803,UNDER,226.5,50.30%
1,Los Angeles Lakers,26.10%,22.536896,-3.563104,UNDER,219,50.20%
2,Memphis Grizzlies,73.90%,70.636597,-3.263403,UNDER,222,50.10%
3,Miami Heat,25.00%,26.431901,1.431901,NaN,NaN,NaN
4,Milwaukee Bucks,75.00%,76.332001,1.332001,NaN,NaN,NaN
5,Minnesota Timberwolves,25.10%,26.631802,1.531802,NaN,NaN,NaN
